# Transfer Learning

In this notebook, you'll learn how to use pre-trained networks to solved challenging problems in computer vision. Specifically, you'll use networks trained on [ImageNet](http://www.image-net.org/) [available from torchvision](http://pytorch.org/docs/0.3.0/torchvision/models.html). 

ImageNet is a massive dataset with over 1 million labeled images in 1000 categories. It's used to train deep neural networks using an architecture called convolutional layers. I'm not going to get into the details of convolutional networks here, but if you want to learn more about them, please [watch this](https://www.youtube.com/watch?v=2-Ol7ZB0MmU).

Once trained, these models work astonishingly well as feature detectors for images they weren't trained on. Using a pre-trained network on images not in the training set is called transfer learning. Here we'll use transfer learning to train a network that can classify our cat and dog photos with near perfect accuracy.

With `torchvision.models` you can download these pre-trained networks and use them in your applications. We'll include `models` in our imports now.

Most of the pretrained models require the input to be 224x224 images. Also, we'll need to match the normalization used when the models were trained. Each color channel was normalized separately, the means are `[0.485, 0.456, 0.406]` and the standard deviations are `[0.229, 0.224, 0.225]`.

We can load in a model such as [DenseNet](http://pytorch.org/docs/0.3.0/torchvision/models.html#id5). Let's print out the model architecture so we can see what's going on.

In [1]:
# model = models.densenet121(pretrained=True)
# model

This model is built out of two main parts, the features and the classifier. The features part is a stack of convolutional layers and overall works as a feature detector that can be fed into a classifier. The classifier part is a single fully-connected layer `(classifier): Linear(in_features=1024, out_features=1000)`. This layer was trained on the ImageNet dataset, so it won't work for our specific problem. That means we need to replace the classifier, but the features will work perfectly on their own. In general, I think about pre-trained networks as amazingly good feature detectors that can be used as the input for simple feed-forward classifiers.

In [2]:
# # Freeze parameters so we don't backprop through them
# for param in model.parameters():
#     param.requires_grad = False

# from collections import OrderedDict
# classifier = nn.Sequential(OrderedDict([
#                           ('fc1', nn.Linear(1024, 500)),
#                           ('relu', nn.ReLU()),
#                           ('fc2', nn.Linear(500, 2)),
#                           ('output', nn.LogSoftmax(dim=1))
#                           ]))
    
# model.classifier = classifier

With our model built, we need to train the classifier. However, now we're using a **really deep** neural network. If you try to train this on a CPU like normal, it will take a long, long time. Instead, we're going to use the GPU to do the calculations. The linear algebra computations are done in parallel on the GPU leading to 100x increased training speeds. It's also possible to train on multiple GPUs, further decreasing training time.

PyTorch, along with pretty much every other deep learning framework, uses [CUDA](https://developer.nvidia.com/cuda-zone) to efficiently compute the forward and backwards passes on the GPU. In PyTorch, you move your model parameters and other tensors to the GPU memory using `model.to('cuda')`. You can move them back from the GPU with `model.to('cpu')` which you'll commonly do when you need to operate on the network output outside of PyTorch. As a demonstration of the increased speed, I'll compare how long it takes to perform a forward and backward pass with and without a GPU.

In [3]:
# import time

In [4]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# criterion = nn.NLLLoss()
# # Only train the classifier parameters, feature parameters are frozen
# # optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

# model.to(device)

# for ii, (inputs, labels) in enumerate(trainloader):

#     # Move input and label tensors to the GPU
#     inputs, labels = inputs.to(device), labels.to(device)

#     start = time.time()

#     outputs = model.forward(inputs)
#     loss = criterion(outputs, labels)
#     loss.backward()
#     optimizer.step()

#     if ii==3:
#         break

# print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

You can write device agnostic code which will automatically use CUDA if it's enabled like so:
```python
# at beginning of the script
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

...

# then whenever you get a new Tensor or Module
# this won't copy if they are already on the desired device
input = data.to(device)
model = MyModule(...).to(device)
```

From here, I'll let you finish training the model. The process is the same as before except now your model is much more powerful. You should get better than 95% accuracy easily.

>**Exercise:** Train a pretrained models to classify the cat and dog images. Continue with the DenseNet model, or try ResNet, it's also a good model to try out first. Make sure you are only training the classifier and the parameters for the features part are frozen.

In [5]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [6]:
data_dir = 'Cat_Dog_data'


# Define transforms for the training data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

# Define transforms for the testing data
test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])

# Upload training data
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)

# Upload testing data
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

# Create training DataLoader, define batch size and shuffle configuration ON
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)

# Create testing DataLreqder, define batch size and shuffle configuration OFF
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [10]:
# Make code agnostic to use CPU or GPU(if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pretreined neural network
model = models.densenet121(pretrained=True)

# We don't want to backpropagate to the trained layers, so we freeze them
for param in model.parameters():
    param.requires_grad = False
    
# create the new last layer to be trained
classifier = nn.Sequential(nn.Linear(1024,256),
                          nn.ReLU(),
                          nn.Dropout(0.2),                           
                          nn.Linear(256,2),
                          nn.LogSoftmax(dim=1))

# replace the original trained neural network classifier layer with the newly create one
model.classifier = classifier

# Choose your loss function
criterion = nn.NLLLoss()

# TODO: Check if this works: optmizer = optim.Adam(parameters)
optmizer = optim.Adam(model.classifier.parameters(), lr=0.003)

# TODO: Check if this works: model.to(device)
model.to(device);

# !! Let's start training !!

# Define number of Epochs
# TODO: test with other epochs
epochs = 1

# Create variables to track number of training steps and running training loss
step = 0
running_loss = 0

# Create variable to print running loss every X steps
print_every = 5

# Train model for "epochs" epochs
for e in range(epochs):
    # get the next batch of training images
    for images, labels in trainloader:
        step += 1
        # Move input and label tensors to the default device
        images, labels = images.to(device), labels.to(device)
        
        # make sure past gradients won't influence this training step
        optmizer.zero_grad()
        
        # do one training step
        output = model.forward(images)
        loss = criterion(output,labels)
        loss.backward()
        optmizer.step()

        # update running loss
        running_loss += loss.item()
        
        # Check if it's time to print model running Loss and Accuracy
        if step % print_every == 0:
            test_losses = 0
            accuracy = 0
            
            # make sure we are in "eval" mode, i.e. turn off drop out
            model.eval()
            
            with torch.no_grad():
                for test_images, test_labels in testloader:
                    # Move input and label tensors to the default device
                    test_images, test_labels = test_images.to(device), test_labels.to(device)

                    # Uses network to calculate output
                    test_output = model.forward(test_images)

                    # Calculate loss produced by network
                    test_loss = criterion(test_output,test_labels)
                    # update testing running loss
                    test_losses += test_loss.item()
            
                    # Calculate accuracy
                    test_output_prob = torch.exp(test_output)
                    top_test_output_prob,top_test_output_class = test_output_prob.topk(1, dim =1)
                    equals = top_test_output_class == test_labels.view(*top_test_output_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            
            print(f"Epoch {e+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_losses/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")

            # remember to set mode to "train" after testing
            model.train()
            
            # reset training running loss cause it's gonna start a new epoch after this
            running_loss = 0
            
            
        
        


Epoch 1/1.. Train loss: 0.712.. Test loss: 0.312.. Test accuracy: 0.877


KeyboardInterrupt: 